## steps 

* prepare data
* design data/model (input,output)
* construct loss and optmizer
* training loop
    - forwar pass: compute prediction and loss
    - backward pass: gradients
    - update weights

In [2]:
import torch 
import torch.nn as nn
import numpy as np
import matplotlib as mtl
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [16]:
#load data
df_store = datasets.load_breast_cancer()

#separate data and target
X, y = df_store.data, df_store.target

#split n_samples, n_features
n_samples, n_features = X.shape[0], X.shape[1]

#train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.3,random_state=2024)

In [19]:
#scale
scaler = StandardScaler()
scaled_train = scaler.fit_transform(X_train)
scaled_test = scaler.transform(X_test)
#torch.from_numpy (include astype)

X_torch_train = torch.from_numpy(scaled_train.astype(np.float32))
X_torch_test = torch.from_numpy(scaled_test.astype(np.float32))
y_torch_train = torch.from_numpy(y_train.astype(np.float32))
y_torch_test = torch.from_numpy(y_test.astype(np.float32))

## A big difference when we want to use torch
#### transform your model in a module